# Tic Tac Toe
Training the program to play tic tac toe. Later there's a possibility to face the computer as a player to see who'll win. At the bottom there's a variable that sets how many times the computer should train. The program learns using Q-learning. Most of this code was given as part of a course I've taken in artificial intelligence and I dont claim any ownership of it.

In [1]:
import pygame
import random
import time

class Humanplayer:
    pass

#randomplayer player
class Randomplayer:
    def __init__(self):
        pass
    def move(self,possiblemoves):
        return random.choice(possiblemoves)

class TicTacToe:
    def __init__(self,traning=False):
        self.board = [' ']*9

        self.done = False
        self.human=None
        self.computer=None
        self.humanTurn=None
        self.training=traning
        self.player1 = None
        self.player2 = None
        self.aiplayer=None
        self.isAI=False
        # if not training display
        if(not self.training):
            pygame.init()
            self.ttt = pygame.display.set_mode((250,250))
            pygame.display.set_caption('Tic-Tac-Toe')

    #reset the game
    def reset(self):
        if(self.training):
            self.board = [' '] * 9
            return

        self.board = [' '] * 9
        self.humanTurn=random.choice([True,False])

        self.surface = pygame.Surface(self.ttt.get_size())
        self.surface = self.surface.convert()
        self.surface.fill((250, 250, 250))
        #horizontal line
        pygame.draw.line(self.surface, (0, 0, 0), (75, 0), (75, 225), 2)
        pygame.draw.line(self.surface, (0, 0, 0), (150, 0), (150, 225), 2)
        # veritical line
        pygame.draw.line(self.surface, (0, 0, 0), (0,75), (225, 75), 2)
        pygame.draw.line(self.surface, (0, 0, 0), (0,150), (225, 150), 2)

   #evaluate function
    def evaluate(self, ch):
        # "rows checking"
        for i in range(3):
            if (ch == self.board[i * 3] == self.board[i * 3 + 1] and self.board[i * 3 + 1] == self.board[i * 3 + 2]):
                return 1.0, True
        # "col checking"
        for i in range(3):
            if (ch == self.board[i + 0] == self.board[i + 3] and self.board[i + 3] == self.board[i + 6]):
                return 1.0, True
        # diagonal checking
        if (ch == self.board[0] == self.board[4] and self.board[4] == self.board[8]):
            return 1.0, True

        if (ch == self.board[2] == self.board[4] and self.board[4] == self.board[6]):
            return 1.0, True
        # "if filled draw"
        if not any(c == ' ' for c in self.board):
            return 0.5, True

        return 0.0, False

    #return remaining possible moves
    def possible_moves(self):
        return [moves + 1 for moves, v in enumerate(self.board) if v == ' ']

    #take next step and return reward
    def step(self, isX, move):
        if(isX):
             ch = 'X'
        else:
            ch = '0'
        if(self.board[move-1]!=' '): # try to over write
            return  -5, True

        self.board[move-1]= ch
        reward,done = self.evaluate(ch)
        return reward, done


    #draw move on window
    def drawMove(self, pos,isX):
        row=int((pos-1)/3)
        col=(pos-1)%3

        centerX = ((col) * 75) + 32
        centerY = ((row) * 75) + 32

        reward, done= self.step(isX,pos) #next step
        if(reward==-5): #overlap
            #print('Invalid move')
            font = pygame.font.Font(None, 18)
            text = font.render('Invalid move! Press any key to reset', 1, (10, 10, 10))
            self.surface.fill((250, 250, 250), (0, 300, 300, 25))
            self.surface.blit(text, (10, 230))

            return reward, done

        if (isX): #playerX so draw x
            font = pygame.font.Font(None, 24)
            text = font.render('X', 1, (10, 10, 10))
            self.surface.fill((250, 250, 250), (0, 300, 300, 25))
            self.surface.blit(text, (centerX, centerY))
            self.board[pos-1] ='X'

            if(self.human and reward==1):
                #print('Human won! in X')
                font = pygame.font.Font(None, 18)
                text = font.render('Human won! Press any key to continue', 1, (10, 10, 10))
                self.surface.fill((250, 250, 250), (0, 300, 300, 25))
                self.surface.blit(text, (10, 230))


            elif (self.computer and reward == 1):#if playerX is computer and won, display computer won
                #print('computer won! in X')
                font = pygame.font.Font(None, 18)
                text = font.render('computer won! Press any key to continue', 1, (10, 10, 10))
                self.surface.fill((250, 250, 250), (0, 300, 300, 25))
                self.surface.blit(text, (10, 230))




        else:  #playerO so draw O
            font = pygame.font.Font(None, 24)
            text = font.render('O', 1, (10, 10, 10))

            self.surface.fill((250, 250, 250), (0, 300, 300, 25))
            self.surface.blit(text, (centerX, centerY))
            self.board[pos-1] = '0'

            if (not self.human and reward == 1):
                font = pygame.font.Font(None, 18)
                text = font.render('Human won!. Press any key to continue', 1, (10, 10, 10))
                self.surface.fill((250, 250, 250), (0, 300, 300, 25))
                self.surface.blit(text, (10, 230))


            elif (not self.computer and reward == 1):  #if playerO is computer and won, display computer won
                #print('computer won! in O')
                font = pygame.font.Font(None, 18)
                text = font.render('Computer won! Press any key to continue', 1, (10, 10, 10))
                self.surface.fill((250, 250, 250), (0, 300, 300, 25))
                self.surface.blit(text, (10, 230))



        if (reward == 0.5):  # draw, then display draw
            #print('Draw Game! in O')
            font = pygame.font.Font(None, 18)
            text = font.render('Draw Game! Press any key to continue', 1, (10, 10, 10))
            self.surface.fill((250, 250, 250), (0, 300, 300, 25))
            self.surface.blit(text, (10, 230))
            return reward, done

        return reward,done

    # mouseClick position
    def mouseClick(self):
        (mouseX, mouseY) = pygame.mouse.get_pos()
        if (mouseY < 75):
            row = 0
        elif (mouseY < 150):
            row = 1
        else:
            row = 2

        if (mouseX < 75):
            col = 0
        elif (mouseX < 150):
            col = 1
        else:
            col = 2
        return row * 3 + col + 1


     #update state
    def updateState(self,isX):
        pos=self.mouseClick()
        reward,done = self.drawMove(pos,isX)
        return reward, done

    #show display
    def showboard(self):
        self.ttt.blit(self.surface, (0, 0))
        pygame.display.flip()


    #begin training
    def startTraining(self,player1,player2):
        if(isinstance(player1,Qlearning) and isinstance(player2, Qlearning)):
            self.training = True
            self.player1=player1
            self.player2=player2

    def train(self,iterations):
        if(self.training):
            for i in range(iterations):
                print("training", i)
                self.player1.game_begin()
                self.player2.game_begin()
                self.reset()
                done = False
                isX = random.choice([True, False])
                while not done:
                    if isX:
                        move = self.player1.epslion_greedy(self.board, self.possible_moves())
                    else:
                        move = self.player2.epslion_greedy(self.board, self.possible_moves())


                    reward, done = self.step(isX, move)

                    if (reward == 1):  # won
                        if (isX):
                            self.player1.updateQ(reward, self.board, self.possible_moves())
                            self.player2.updateQ(-1 * reward, self.board, self.possible_moves())
                        else:
                            self.player1.updateQ(-1 * reward, self.board, self.possible_moves())
                            self.player2.updateQ(reward, self.board, self.possible_moves())

                    elif (reward == 0.5):  # draw
                        self.player1.updateQ(reward, self.board, self.possible_moves())
                        self.player2.updateQ(reward, self.board, self.possible_moves())


                    elif (reward == -5):  # illegal move
                        if (isX):
                            self.player1.updateQ(reward, self.board, self.possible_moves())
                        else:
                            self.player2.updateQ(reward, self.board, self.possible_moves())

                    elif (reward == 0):
                        if (isX):  # update opposite
                            self.player2.updateQ(reward, self.board, self.possible_moves())
                        else:
                            self.player1.updateQ(reward, self.board, self.possible_moves())

                    isX = not isX  #

    #save Qtables
    def saveStates(self):
        self.player1.saveQtable("player1states")
        self.player2.saveQtable("player2states")


    #start game human vs AI or human vs random
    def startGame(self, playerX, playerO):
        if (isinstance(playerX, Humanplayer)):
            self.human, self.computer = True, False
            if (isinstance(playerO, Qlearning)): #if AI
                self.ai = playerO
                self.ai.loadQtable("player2states") # load saved Q table
                self.ai.epsilon = 0 #set eps to 0 so always choose greedy step
                self.isAI = True
            elif (isinstance(playerO, Randomplayer)): #if random
                self.ai = playerO
                self.isAI = False

        elif (isinstance(playerO, Humanplayer)):
            self.human, self.computer = False, True
            if (isinstance(playerX, Qlearning)): #if AI
                self.ai = playerX
                self.ai.loadQtable("player1states") # load saved Q table
                self.ai.epsilon = 0 #set eps to 0 so always choose greedy step
                self.isAI = True
            elif(isinstance(playerX, Randomplayer)):#if random
                self.ai=playerX
                self.isAI = False


    def render(self):
        running = 1
        done = False
        pygame.event.clear()
        while (running == 1):
            if (self.humanTurn):
                print("Human player turn")
                event = pygame.event.wait()
                while event.type != pygame.MOUSEBUTTONDOWN:
                    event = pygame.event.wait()
                    self.showboard()
                    if event.type == pygame.QUIT:
                        running = 0
                        print("pressed quit")
                        pygame.display.quit()
                        break
                if running == 1:
                    reward, done = self.updateState(self.human) #if random
                    self.showboard()
                    if (done): #if done reset
                        #time.sleep(1)
                        while event.type != pygame.KEYDOWN:
                            event = pygame.event.wait()
                            self.showboard()
                            if event.type == pygame.QUIT:
                                running = 0
                                print("pressed quit")
                                pygame.display.quit()
                                break
                        pygame.event.clear()
                        self.reset()
                        
            else:  #AI or random turn
                if(self.isAI):
                    moves = self.ai.epslion_greedy(self.board, self.possible_moves())
                    reward, done = self.drawMove(moves, self.computer)
                    print("computer's AI player turn")
                    self.showboard()
                else: #random player
                    moves = self.ai.move(self.possible_moves()) #random player
                    reward, done = self.drawMove(moves, self.computer)
                    print("computer's random player turn")
                    self.showboard()

                if (done): #if done reset
                    #time.sleep(1)
                    while event.type != pygame.KEYDOWN:
                        event = pygame.event.wait()
                        self.showboard()
                        if event.type == pygame.QUIT:
                            running = 0
                            print("pressed quit")
                            pygame.display.quit()
                            break
                    pygame.event.clear()
                    self.reset()
                    

            self.humanTurn = not self.humanTurn

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import random
import pickle

class Qlearning:
    def __init__(self,epsilon=0.2, alpha=0.3, gamma=0.9):
        self.epsilon=epsilon
        self.alpha=alpha
        self.gamma=gamma
        self.Q = {} #Q table
        self.last_board=None
        self.q_last=0.0
        self.state_action_last=None

    def game_begin(self):
        self.last_board = None
        self.q_last = 0.0
        self.state_action_last = None


    def epslion_greedy(self, state, possible_moves): #esplion greedy algorithm
        #return  action
        self.last_board = tuple(state)
        if(random.random() < self.epsilon):
            move = random.choice(possible_moves) ##action
            self.state_action_last=(self.last_board,move)
            self.q_last=self.getQ(self.last_board,move)
            return move
        else: #greedy strategy
            Q_list=[]
            for action in possible_moves:
                Q_list.append(self.getQ(self.last_board,action))
            maxQ=max(Q_list)

            if Q_list.count(maxQ) > 1:
                # more than 1 best option; choose among them randomly
                best_options = [i for i in range(len(possible_moves)) if Q_list[i] == maxQ]
                i = random.choice(best_options)
            else:
                i = Q_list.index(maxQ)
            self.state_action_last = (self.last_board, possible_moves[i])
            self.q_last = self.getQ(self.last_board, possible_moves[i])
            return possible_moves[i]


    def getQ(self, state, action): #get Q states
        if(self.Q.get((state,action))) is None:
            self.Q[(state,action)] = 1.0
        return self.Q.get((state,action))

    def updateQ(self, reward, state, possible_moves): # update Q states using Qleanning
        q_list=[]
        for moves in possible_moves:
            q_list.append(self.getQ(tuple(state), moves))
        if q_list:
            max_q_next = max(q_list)
        else:
            max_q_next=0.0
        self.Q[self.state_action_last] = self.q_last + self.alpha * ((reward + self.gamma*max_q_next) - self.q_last)

    def saveQtable(self,file_name):  #save table
        with open(file_name, 'wb') as handle:
            pickle.dump(self.Q, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def loadQtable(self,file_name): # load table
        with open(file_name, 'rb') as handle:
            self.Q = pickle.load(handle)


In [3]:
#Training
numiter = 25000
game = TicTacToe(True) #game instance, True means training
player1= Qlearning() #player1 learning agent
player2 =Qlearning() #player2 learning agent
game.startTraining(player1,player2) #start training
game.train(numiter) #train for 200,000 iterations
game.saveStates()  #save Qtable

game = TicTacToe() #game instance
player1=Humanplayer() #human player
player2=Qlearning()  #agent
game.startGame(player1,player2)#player1 is X, player2 is 0
game.reset() #reset
game.render() # render display

trainining 0
trainining 1
trainining 2
trainining 3
trainining 4
trainining 5
trainining 6
trainining 7
trainining 8
trainining 9
trainining 10
trainining 11
trainining 12
trainining 13
trainining 14
trainining 15
trainining 16
trainining 17
trainining 18
trainining 19
trainining 20
trainining 21
trainining 22
trainining 23
trainining 24
trainining 25
trainining 26
trainining 27
trainining 28
trainining 29
trainining 30
trainining 31
trainining 32
trainining 33
trainining 34
trainining 35
trainining 36
trainining 37
trainining 38
trainining 39
trainining 40
trainining 41
trainining 42
trainining 43
trainining 44
trainining 45
trainining 46
trainining 47
trainining 48
trainining 49
trainining 50
trainining 51
trainining 52
trainining 53
trainining 54
trainining 55
trainining 56
trainining 57
trainining 58
trainining 59
trainining 60
trainining 61
trainining 62
trainining 63
trainining 64
trainining 65
trainining 66
trainining 67
trainining 68
trainining 69
trainining 70
trainining 71
tr

trainining 610
trainining 611
trainining 612
trainining 613
trainining 614
trainining 615
trainining 616
trainining 617
trainining 618
trainining 619
trainining 620
trainining 621
trainining 622
trainining 623
trainining 624
trainining 625
trainining 626
trainining 627
trainining 628
trainining 629
trainining 630
trainining 631
trainining 632
trainining 633
trainining 634
trainining 635
trainining 636
trainining 637
trainining 638
trainining 639
trainining 640
trainining 641
trainining 642
trainining 643
trainining 644
trainining 645
trainining 646
trainining 647
trainining 648
trainining 649
trainining 650
trainining 651
trainining 652
trainining 653
trainining 654
trainining 655
trainining 656
trainining 657
trainining 658
trainining 659
trainining 660
trainining 661
trainining 662
trainining 663
trainining 664
trainining 665
trainining 666
trainining 667
trainining 668
trainining 669
trainining 670
trainining 671
trainining 672
trainining 673
trainining 674
trainining 675
trainining

trainining 1236
trainining 1237
trainining 1238
trainining 1239
trainining 1240
trainining 1241
trainining 1242
trainining 1243
trainining 1244
trainining 1245
trainining 1246
trainining 1247
trainining 1248
trainining 1249
trainining 1250
trainining 1251
trainining 1252
trainining 1253
trainining 1254
trainining 1255
trainining 1256
trainining 1257
trainining 1258
trainining 1259
trainining 1260
trainining 1261
trainining 1262
trainining 1263
trainining 1264
trainining 1265
trainining 1266
trainining 1267
trainining 1268
trainining 1269
trainining 1270
trainining 1271
trainining 1272
trainining 1273
trainining 1274
trainining 1275
trainining 1276
trainining 1277
trainining 1278
trainining 1279
trainining 1280
trainining 1281
trainining 1282
trainining 1283
trainining 1284
trainining 1285
trainining 1286
trainining 1287
trainining 1288
trainining 1289
trainining 1290
trainining 1291
trainining 1292
trainining 1293
trainining 1294
trainining 1295
trainining 1296
trainining 1297
trainini

trainining 1945
trainining 1946
trainining 1947
trainining 1948
trainining 1949
trainining 1950
trainining 1951
trainining 1952
trainining 1953
trainining 1954
trainining 1955
trainining 1956
trainining 1957
trainining 1958
trainining 1959
trainining 1960
trainining 1961
trainining 1962
trainining 1963
trainining 1964
trainining 1965
trainining 1966
trainining 1967
trainining 1968
trainining 1969
trainining 1970
trainining 1971
trainining 1972
trainining 1973
trainining 1974
trainining 1975
trainining 1976
trainining 1977
trainining 1978
trainining 1979
trainining 1980
trainining 1981
trainining 1982
trainining 1983
trainining 1984
trainining 1985
trainining 1986
trainining 1987
trainining 1988
trainining 1989
trainining 1990
trainining 1991
trainining 1992
trainining 1993
trainining 1994
trainining 1995
trainining 1996
trainining 1997
trainining 1998
trainining 1999
trainining 2000
trainining 2001
trainining 2002
trainining 2003
trainining 2004
trainining 2005
trainining 2006
trainini

trainining 2485
trainining 2486
trainining 2487
trainining 2488
trainining 2489
trainining 2490
trainining 2491
trainining 2492
trainining 2493
trainining 2494
trainining 2495
trainining 2496
trainining 2497
trainining 2498
trainining 2499
trainining 2500
trainining 2501
trainining 2502
trainining 2503
trainining 2504
trainining 2505
trainining 2506
trainining 2507
trainining 2508
trainining 2509
trainining 2510
trainining 2511
trainining 2512
trainining 2513
trainining 2514
trainining 2515
trainining 2516
trainining 2517
trainining 2518
trainining 2519
trainining 2520
trainining 2521
trainining 2522
trainining 2523
trainining 2524
trainining 2525
trainining 2526
trainining 2527
trainining 2528
trainining 2529
trainining 2530
trainining 2531
trainining 2532
trainining 2533
trainining 2534
trainining 2535
trainining 2536
trainining 2537
trainining 2538
trainining 2539
trainining 2540
trainining 2541
trainining 2542
trainining 2543
trainining 2544
trainining 2545
trainining 2546
trainini

...

trainining 22973
trainining 22974
trainining 22975
trainining 22976
trainining 22977
trainining 22978
trainining 22979
trainining 22980
trainining 22981
trainining 22982
trainining 22983
trainining 22984
trainining 22985
trainining 22986
trainining 22987
trainining 22988
trainining 22989
trainining 22990
trainining 22991
trainining 22992
trainining 22993
trainining 22994
trainining 22995
trainining 22996
trainining 22997
trainining 22998
trainining 22999
trainining 23000
trainining 23001
trainining 23002
trainining 23003
trainining 23004
trainining 23005
trainining 23006
trainining 23007
trainining 23008
trainining 23009
trainining 23010
trainining 23011
trainining 23012
trainining 23013
trainining 23014
trainining 23015
trainining 23016
trainining 23017
trainining 23018
trainining 23019
trainining 23020
trainining 23021
trainining 23022
trainining 23023
trainining 23024
trainining 23025
trainining 23026
trainining 23027
trainining 23028
trainining 23029
trainining 23030
trainining 230

trainining 23722
trainining 23723
trainining 23724
trainining 23725
trainining 23726
trainining 23727
trainining 23728
trainining 23729
trainining 23730
trainining 23731
trainining 23732
trainining 23733
trainining 23734
trainining 23735
trainining 23736
trainining 23737
trainining 23738
trainining 23739
trainining 23740
trainining 23741
trainining 23742
trainining 23743
trainining 23744
trainining 23745
trainining 23746
trainining 23747
trainining 23748
trainining 23749
trainining 23750
trainining 23751
trainining 23752
trainining 23753
trainining 23754
trainining 23755
trainining 23756
trainining 23757
trainining 23758
trainining 23759
trainining 23760
trainining 23761
trainining 23762
trainining 23763
trainining 23764
trainining 23765
trainining 23766
trainining 23767
trainining 23768
trainining 23769
trainining 23770
trainining 23771
trainining 23772
trainining 23773
trainining 23774
trainining 23775
trainining 23776
trainining 23777
trainining 23778
trainining 23779
trainining 237

trainining 24472
trainining 24473
trainining 24474
trainining 24475
trainining 24476
trainining 24477
trainining 24478
trainining 24479
trainining 24480
trainining 24481
trainining 24482
trainining 24483
trainining 24484
trainining 24485
trainining 24486
trainining 24487
trainining 24488
trainining 24489
trainining 24490
trainining 24491
trainining 24492
trainining 24493
trainining 24494
trainining 24495
trainining 24496
trainining 24497
trainining 24498
trainining 24499
trainining 24500
trainining 24501
trainining 24502
trainining 24503
trainining 24504
trainining 24505
trainining 24506
trainining 24507
trainining 24508
trainining 24509
trainining 24510
trainining 24511
trainining 24512
trainining 24513
trainining 24514
trainining 24515
trainining 24516
trainining 24517
trainining 24518
trainining 24519
trainining 24520
trainining 24521
trainining 24522
trainining 24523
trainining 24524
trainining 24525
trainining 24526
trainining 24527
trainining 24528
trainining 24529
trainining 245

computer's AI player turn
Human player turn
computer's AI player turn
Human player turn
computer's AI player turn
Human player turn
computer's AI player turn
pressed quit


error: video system not initialized